In [5]:
import pandas as pd
import os 
from dotenv import load_dotenv
import logging
import os
from flask import current_app
import time
import glob
import json

import gspread
from google.oauth2.service_account import Credentials

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [6]:
source_amz = pd.read_excel('data/mapping source/Amazon.xlsx', sheet_name='Master Sku Translation', dtype=str)

amz_sku_mapping = source_amz[['seller-sku', 'SKU (FINAL)', 'Product','Product NZ']].copy()
amz_sku_mapping = amz_sku_mapping.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
# rename columns to ['seller-sku', 'BS_SKU', 'B_SKU','B_NZ_SKU']
amz_sku_mapping.columns = ['seller_sku', 'BS_SKU', 'B_SKU','B_NZ_SKU']





In [7]:
def read_files(directory):
    data = {}
    for filename in os.listdir(directory):
        if filename.endswith(".csv"):
            name, _ = os.path.splitext(filename)
            data[name] = pd.read_csv(os.path.join(directory, filename))
    return data

In [8]:
#preparing/data/walmart/CA.csv

dfs = read_files('data/Walmart/')

# def combine_all_listings_by_SKU_Walmart(data_dict):
#     """
#     This function combines Walmart product listings from multiple countries into a single DataFrame.
#     Each country's DataFrame is processed to retain only the 'Supplier Part#' and 'Supplier ID' columns.
#     The 'Supplier ID' column is renamed to indicate the country of origin.
#     The processed DataFrames are then merged on the 'Supplier Part#' column.

#     Parameters:
#     data_dict (dict): A dictionary where keys are country names and values are DataFrames containing Walmart product listings.

#     Returns:
#     result_df (pandas.DataFrame): A DataFrame containing the combined product listings. Each row represents a unique product listing.
#     The DataFrame contains columns for 'Supplier Part#' and 'Supplier ID_{country}' for each country.
#     """

#     def process_country(df, country):
#         df = df[['Supplier Part#', 'Supplier ID']].copy()
#         df.columns = ['seller_sku', f'Supplier_ID_{country}']
#         return df

#     processed_dfs = []
#     for country, df in data_dict.items():
#         country_code = country.split('_')[0].upper()  # Assuming filename format is like "US_..."
#         processed_dfs.append(process_country(df, country_code))

#     result_df = processed_dfs[0]
#     for df in processed_dfs[1:]:
#         result_df = pd.merge(result_df, df, on='seller_sku', how='outer')

#     return result_df


# combined_df = combine_all_listings_by_SKU_Walmart(dfs)


CA_df = dfs['CA']
US_df = dfs['US']

US_df = US_df[['SKU', 'Item ID']]
US_df.columns = ['seller_sku', 'item_id_US']

CA_df = CA_df[['Seller Sku ID', 'Walmart SKU ID']]
CA_df.columns = ['seller_sku', 'item_id_CA']

combined_df = pd.merge(US_df, CA_df, on='seller_sku', how='outer')

In [9]:


final_NA_mapping = pd.merge(combined_df, amz_sku_mapping, on='seller_sku', how='left')

final_NA_mapping.to_csv('results/Walmart_sku_mapping.csv', index=False)


In [10]:

def update_sheet_in_chunks(worksheet, df, chunk_size=2000, continue_from_end=False, start_row=0):
    try:
        logger.info(f'Updating google sheet "{worksheet }" in chunks')
        header = worksheet.row_values(1)
        df_coloumns_list = df.columns.values.tolist()

        if not continue_from_end:
            worksheet.clear()
            start_row = 0
        else:
            if header != df_coloumns_list:
                logger.error(f'updating google sheet "{worksheet}" Header mismatch: {header} != {df_coloumns_list}')
                raise Exception(f'updating google sheet "{worksheet}" Header mismatch: {header} != {df_coloumns_list}')
                
        worksheet_length = worksheet.row_count
        df_columns_length = len(df.columns)

        ensure_sheet_size(worksheet, worksheet_length + len(df),df_columns_length )

        df_list = df_to_sheet_list(df, header= not continue_from_end)
         
        for i in range(start_row, len(df_list) + start_row, chunk_size):
            chunk = df_list[i:i+chunk_size]
            range_name = f'A{i+1}:{column_number_to_letter(df_columns_length)}{i+len(chunk)}'
            logger.info(f'Updating chunk {range_name}')

            worksheet.update(chunk, range_name)
            # add timout to avoid google api rate limit
            time.sleep(0.3)
    except Exception as e:
        logger.error(f'Error updating sheet in chunks: {str(e)}')


def ensure_sheet_size(worksheet, required_rows, required_cols):
    current_rows = worksheet.row_count
    current_cols = worksheet.col_count

    if required_rows > current_rows or required_cols > current_cols:
        new_rows = max(required_rows, current_rows)
        new_cols = max(required_cols, current_cols)
        worksheet.resize(new_rows, new_cols)

def df_to_sheet_list(df, header=True):
    df =df.fillna('')
    df = df.replace({pd.NA: ''})
    df = df.replace({'nan': ''})
    if header:
        df = [df.columns.values.tolist()] + df.values.tolist()
    else:
        df = df.values.tolist()
    return df

def column_number_to_letter(n):
    string = ""
    while n > 0:
        n, remainder = divmod(n - 1, 26)
        string = chr(65 + remainder) + string
    return string


def get_workbook(sheet_id = "1ZMzIMn7CzV_tUJSfXguHYLh3fkkgHVh_0u2NBWCzEAQ"):
    scopes = ["https://www.googleapis.com/auth/spreadsheets"]
    creds = get_sheets_api_credentials()
    client = gspread.authorize(creds)
    workbook = client.open_by_key(sheet_id)
    return workbook


def get_sheets_api_credentials():
    # Load environment variables
    load_dotenv(override=True)
    logger.info("Getting google sheets api credentials")
    # Create a dictionary with the credentials
    cred_dict = {
        "type": "service_account",
        "project_id": "w4l-inventory-update",
        "private_key_id": os.getenv("GOOGLE_PRIVATE_KEY_ID"),
        "private_key": os.getenv("GOOGLE_PRIVATE_KEY"),   
        "client_email": os.getenv("GOOGLE_CLIENT_EMAIL"),
        "client_id": os.getenv("GOOGLE_CLIENT_ID"),
        "auth_uri": "https://accounts.google.com/o/oauth2/auth",
        "token_uri": "https://oauth2.googleapis.com/token",
        "auth_provider_x509_cert_url": "https://www.googleapis.com/oauth2/v1/certs",
        "client_x509_cert_url": os.getenv("GOOGLE_CLIENT_X509_CERT_URL"),
        "universe_domain": "googleapis.com"
    }

    # Create a temporary file to store the credentials
    temp_cred_file = 'temp_credentials.json'
    # if temp_cred_file is exist delate it first 
    if os.path.exists(temp_cred_file):
        os.remove(temp_cred_file)
        logger.info("Deleted the existing temp_cred_file")

    with open(temp_cred_file, 'w') as f:
        json.dump(cred_dict, f)
        logger.info("Created the temp_cred_file")


    # Get the credentials from the temporary file
    scopes = ["https://www.googleapis.com/auth/spreadsheets"]
    
    creds = Credentials.from_service_account_file(temp_cred_file, scopes=scopes)

    # Remove the temporary file
    os.remove(temp_cred_file)

    return creds




def get_worksheet_df_by_name(workbook,worksheet_name):

    worksheet_list = map(lambda x: x.title, workbook.worksheets())
    if worksheet_name not in worksheet_list:
        logging.error(f"Worksheet {worksheet_name} not found in the google sheet")
        raise ValueError(f"Worksheet {worksheet_name} not found in the google sheet")
    # get the worksheet to df 
    return workbook.worksheet(worksheet_name)

def a_ph(relative_path):
    """
    Creates an absolute path based on the relative path from the project root.
    
    :param relative_path: Relative path from the project root
    :return: Absolute path
    """
    if current_app:
        # If the function is called within the context of a Flask application
        root_path = current_app.root_path
    else:
        # If the function is called outside the context of a Flask application
        # Get the parent directory of the directory containing this file
        root_path = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
    
    # Join the root path with the relative path, removing any leading '/'
    return os.path.join(root_path, relative_path.lstrip('/'))

# Example usage:
# print(a_ph('config/settings.json'))
# print(a_ph('/static/images/logo.png'))

def is_inv_updated_today():
    # gen name of the download/update_files_07_18_24.zip 
    # extract the date from the name
    # check if the date is today
    # return True or False

    zip_files = glob.glob(a_ph('/download/*.zip'))
    if not zip_files:
        return False
    else:
        zip_file = os.path.basename(zip_files[0])
        date_str = zip_file.replace('update_files_', '').replace('.zip', '')
        today = time.strftime('%m_%d_%y')
        return date_str == today

In [11]:

# update worksheet  with final_NA_mapping
workbook = get_workbook()
worksheet = get_worksheet_df_by_name(workbook, "walmart_sku_mapping")
update_sheet_in_chunks(worksheet, final_NA_mapping, chunk_size=2000, continue_from_end=False, start_row=0)

INFO:__main__:Getting google sheets api credentials
INFO:__main__:Created the temp_cred_file
INFO:__main__:Updating google sheet "<Worksheet 'walmart_sku_mapping' id:1392378980>" in chunks
INFO:__main__:Updating chunk A1:F2000
INFO:__main__:Updating chunk A2001:F4000
INFO:__main__:Updating chunk A4001:F6000
INFO:__main__:Updating chunk A6001:F8000
INFO:__main__:Updating chunk A8001:F10000
INFO:__main__:Updating chunk A10001:F12000
INFO:__main__:Updating chunk A12001:F14000
INFO:__main__:Updating chunk A14001:F14988
